In [11]:
# Conection to PostgreSQL in RDS
from sqlalchemy import create_engine
import pandas as pd

db_user = 'master'
db_password = 'xxxxx'
db_host = 'disaster-tweets-db.cjkms2wu4u6j.eu-north-1.rds.amazonaws.com'  # sin comillas dobles alrededor
db_port = '5432'
db_name = 'disaster-tweets-db'

engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Load data from training table
df_train = pd.read_sql('SELECT * FROM tweets_train;', engine)

# Check first rows
df_train.head()

,id,keyword,location,text,target
0,1,None,None,Our Deeds are the Reason of this #earthquake M...,1
1,4,None,None,Forest fire near La Ronge Sask. Canada,1
2,5,None,None,All residents asked to 'shelter in place' are ...,1
3,6,None,None,"13,000 people receive #wildfires evacuation or...",1
4,7,None,None,Just got sent this photo from Ruby #Alaska as ...,1


### 1. ETL -> basic cleaning pipeline & textual preprocess

In [3]:
print(df_train['text'].sample(5).values) # check some samples
df_train_etl = df_train.copy() # create a copy to work with

['@themagickidraps not upset with a rally upset with burning buildings businesses executing cops that have nothing to do with it etc'
 'I get to smoke my shit in peace'
 "1st wk of Rainier Diet and my street Seward Park Ave is inundated w/ bypass traffic so @seattledot what's your plan? @seattletimes"
 'The EFAK would be designed for building occupants once they evacuate and report to their evacuation assembly sites'
 '#TeamFollowBack Madhya Pradesh Train Derailment: Village Youth Saved Many Lives  #FollowBack']


In [4]:
# function to clean text

import re
import string

def clean_text(text):
    text = text.lower() # - convert to lowercase
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # - delete urls
    text = re.sub(r'\@\w+|\#', '', text)  # - delete mentions f.e. @user and hashtags f.e. #hashtag
    text = re.sub(r'[0-9]+', '', text)  # - delete numbers
    text = re.sub(rf"[{re.escape(string.punctuation)}]", ' ', text)  # - delete punctuation marks
    text = re.sub(r'\s+', ' ', text).strip()  # - delete special characters and multiple spaces
    return text

df_train_etl['text_clean'] = df_train_etl['text'].apply(clean_text)
df_train_etl.head()

,id,keyword,location,text,target,text_clean
0,1,None,None,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...
1,4,None,None,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,None,None,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...
3,6,None,None,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in ...
4,7,None,None,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...


In [5]:
df_train_etl['text_len'] = df_train_etl['text_clean'].apply(len)
df_train_etl['word_count'] = df_train_etl['text_clean'].apply(lambda x: len(x.split()))
df_train_etl.head()

,id,keyword,location,text,target,text_clean,text_len,word_count
0,1,None,None,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...,68,13
1,4,None,None,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,37,7
2,5,None,None,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...,130,22
3,6,None,None,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in ...,56,7
4,7,None,None,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...,85,16


In [6]:
# study to decide if text from hashtags are kept or deleted
hashtags_disasters = df_train_etl[(df_train_etl['target'] == 1) & (df_train_etl['text'].str.contains(r'#\w+', regex=True))]

from collections import Counter
hashtags = []

for tweet in hashtags_disasters['text']:
    hashtags += re.findall(r'#\w+', tweet.lower())

# Contar frecuencia
hashtag_counts = Counter(hashtags)

# Mostrar los 10 más frecuentes
print(hashtag_counts.most_common(10))

# conclusion: analysing more frequent hashtags when target=1 vs. target=0, we observe quite different ones, so they could have enought influence
# on target to keep text from hashtags (so we only delete symbol "#") as we've applied on previous function "clean_text"

[('#news', 56), ('#hiroshima', 22), ('#earthquake', 19), ('#hot', 13), ('#prebreak', 13), ('#best', 13), ('#japan', 11), ('#india', 10), ('#yyc', 10), ('#breaking', 9)]


In [7]:
# lemmatization -> applying grammatical dictionaries are used to get the right base shape

import spacy
from tqdm import tqdm

# loading english language model from spaCy
nlp = spacy.load("en_core_web_sm")

# function to lemmatize a text
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

# Aplicamos la lematización con una barra de progreso
tqdm.pandas()
df_train_etl['text_lemmatized'] = df_train_etl['text_clean'].progress_apply(lemmatize_text)
df_train_etl.head()

100%|█████████████████████████████████████████████████████████████████████████████| 7613/7613 [00:43<00:00, 174.53it/s]


,id,keyword,location,text,target,text_clean,text_len,word_count,text_lemmatized
0,1,None,None,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...,68,13,our deed be the reason of this earthquake may ...
1,4,None,None,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,37,7,forest fire near la ronge sask canada
2,5,None,None,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...,130,22,all resident ask to shelter in place be be not...
3,6,None,None,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in ...,56,7,people receive wildfire evacuation order in ca...
4,7,None,None,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...,85,16,just got send this photo from ruby alaska as s...


In [8]:
# transformations on keyword column

# overview
print(df_train_etl['keyword'].isnull().sum())
print(df_train_etl['keyword'].nunique())
print(df_train_etl['keyword'].value_counts().head(10))

df_train_etl['keyword'] = df_train_etl['keyword'].fillna('no_keyword') # replace NaN with "no_keyword"

# new column replacing "_" and "%20" with " " & tranforming to lowercase
df_train_etl['keyword_clean'] = df_train_etl['keyword'].str.replace('_', ' ').str.replace('%20',' ').str.lower()

df_train_etl['keyword_clean'] = df_train_etl['keyword_clean'].str.replace('\s+', ' ', regex=True).str.strip() # clean possible multiple spaces

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_train_etl['keyword_encoded'] = le.fit_transform(df_train_etl['keyword_clean']) # new column with encoded keyword (assigning numbers)

# check keywords vs. target relation: ones with more target=1 and ones with more target=0
keyword_target_corr = df_train_etl.groupby('keyword_clean')['target'].mean().sort_values(ascending=False)
print(keyword_target_corr.head(10))  # more associated to disasters
print(keyword_target_corr.tail(10))  # less associated to disasters

61
221
fatalities     45
deluge         42
armageddon     42
harm           41
body%20bags    41
damage         41
sinking        41
twister        40
siren          40
evacuate       40
Name: keyword, dtype: int64
keyword_clean
wreckage           1.000000
derailment         1.000000
debris             1.000000
outbreak           0.975000
typhoon            0.973684
oil spill          0.973684
suicide bombing    0.969697
suicide bomber     0.967742
bombing            0.931034
suicide bomb       0.914286
Name: target, dtype: float64
keyword_clean
blew up        0.060606
panicking      0.060606
traumatised    0.057143
screaming      0.055556
electrocute    0.031250
body bag       0.030303
blazing        0.029412
ruin           0.027027
body bags      0.024390
aftershock     0.000000
Name: target, dtype: float64


In [9]:
# transformations on location column

# fill NaN with 'unknown'
df_train_etl['location'] = df_train_etl['location'].fillna('unknown')

# Normalizar: minúsculas, quitar espacios extra y caracteres no alfabéticos
import re

def clean_location(text):
    text = text.lower() # tranforming to lowercase
    text = re.sub(r'[^a-z\s]', '', text)  # deleting all except letters & spaces
    text = re.sub(r'\s+', ' ', text).strip() # delete multiples spaces
    return text

df_train_etl['location_clean'] = df_train_etl['location'].apply(clean_location)

# check locations more related with target
from collections import Counter

# Top 10 locations more common by target
top_loc_target_1 = Counter(df_train_etl[df_train_etl['target'] == 1]['location_clean']).most_common(10)
top_loc_target_0 = Counter(df_train_etl[df_train_etl['target'] == 0]['location_clean']).most_common(10)

print("🔴 Top locations with target = 1:")
for loc, count in top_loc_target_1:
    print(f"{loc}: {count}")

print("\n🔵 Top locations with target = 0:")
for loc, count in top_loc_target_0:
    print(f"{loc}: {count}")

# Calculate target mean by location
loc_stats = (
    df_train_etl.groupby('location_clean')['target']
    .agg(['count', 'mean'])
    .rename(columns={'count': 'tweet_count', 'mean': 'disaster_ratio'})
    .sort_values(by='disaster_ratio', ascending=False)
)

# Show 10 locations with higher probability to be asociated to natural disasters (target=1)
print("\n🌍 Locations with higher % of target=1:")
print(loc_stats.head(10))

# Same as before but filtering with >5 tweets per location
location_stats_filtered = loc_stats[loc_stats['tweet_count'] >= 5]
print("\n🌍 Locations with higher % of target=1 and >5 tweets:")
print(location_stats_filtered.head(10))

# conclusions after checking results from previous analysis on location column:
# - masive presence of "unknown" values (with target=0 and target=1)
# - name inconsistencies, f.e. usa/united states/u.s.a; solid geographic base would be necessary
# - almost all locations with target=1 on all tweets have <5 tweets; it doesn't give representation/generalization
# - model should face new locations after training, so location codification would be useless
# SO... It isn't worth using this feature as a source of the model to be applied

🔴 Top locations with target = 1:
unknown: 1076
usa: 70
: 40
washington dc: 29
nigeria: 28
united states: 27
india: 22
uk: 19
worldwide: 19
mumbai: 19

🔵 Top locations with target = 0:
unknown: 1460
: 65
new york: 60
usa: 39
london: 33
united states: 23
los angeles ca: 20
canada: 17
kenya: 16
worldwide: 14

🌍 Locations with higher % of target=1:
                   tweet_count  disaster_ratio
location_clean                                
zimbabwe                     1             1.0
fiore lamia scale            1             1.0
own planet                   1             1.0
oxford ms                    1             1.0
flipadelphia                 1             1.0
pa usa                       1             1.0
pacific northwest            1             1.0
paducah ky                   1             1.0
finland                      1             1.0
over the moon                1             1.0

🌍 Locations with higher % of target=1 and >5 tweets:
                          tweet_cou

In [ ]:
# export clean training data to database in RDS instance
df_train_etl.to_sql('tweets_train_clean', engine, if_exists='replace', index=False)

In [12]:
# check table has been created and its content
from sqlalchemy import inspect

# Check if new table is defined at database in RDS instance
inspector = inspect(engine)
print(inspector.get_table_names())

df_check = pd.read_sql('SELECT * FROM tweets_train_clean LIMIT 5', con=engine)
df_check.head()

['tweets_train', 'tweets_test', 'tweets_submission', 'tweets_train_clean']


,id,keyword,location,text,target,text_clean,text_len,word_count,text_lemmatized,keyword_clean,keyword_encoded,location_clean
0,1,no_keyword,unknown,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...,68,13,our deed be the reason of this earthquake may ...,no keyword,147,unknown
1,4,no_keyword,unknown,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,37,7,forest fire near la ronge sask canada,no keyword,147,unknown
2,5,no_keyword,unknown,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...,130,22,all resident ask to shelter in place be be not...,no keyword,147,unknown
3,6,no_keyword,unknown,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in ...,56,7,people receive wildfire evacuation order in ca...,no keyword,147,unknown
4,7,no_keyword,unknown,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...,85,16,just got send this photo from ruby alaska as s...,no keyword,147,unknown


### 2 - Apply same pipeline to test.csv (for submission)

In [15]:
# Load data from test table
df_test = pd.read_sql("SELECT * FROM tweets_test", engine)

df_test_etl = df_test.copy() # create a copy to work with
df_test_etl['text_clean'] = df_test_etl['text'].apply(clean_text) # function to clean text

df_test_etl['text_len'] = df_test_etl['text_clean'].apply(len) # create column with lenght
df_test_etl['word_count'] = df_test_etl['text_clean'].apply(lambda x: len(x.split())) # create column with word count

tqdm.pandas()
df_test_etl['text_lemmatized'] = df_test_etl['text_clean'].progress_apply(lemmatize_text) # apply lemmatization to clean_text

# transformations on keyword column
df_test_etl['keyword'] = df_test_etl['keyword'].fillna('no_keyword') # replace NaN with "no_keyword"
# new column replacing "_" and "%20" with " " & tranforming to lowercase
df_test_etl['keyword_clean'] = df_test_etl['keyword'].str.replace('_', ' ').str.replace('%20',' ').str.lower()
df_test_etl['keyword_clean'] = df_test_etl['keyword_clean'].str.replace('\s+', ' ', regex=True).str.strip() # clean possible multiple spaces
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_test_etl['keyword_encoded'] = le.fit_transform(df_test_etl['keyword_clean']) # new column with encoded keyword (assigning numbers)

# transformations on location column
df_test_etl['location'] = df_test_etl['location'].fillna('unknown') # fill NaN with 'unknown'
df_test_etl['location_clean'] = df_test_etl['location'].apply(clean_location)

# export clean test data to database in RDS instance
df_test_etl.to_sql('tweets_test_clean', engine, if_exists='replace', index=False)

# check table has been created and its content
from sqlalchemy import inspect
inspector = inspect(engine) # Check if new table is defined at database in RDS instance
print(inspector.get_table_names())
df_check_test = pd.read_sql('SELECT * FROM tweets_test_clean LIMIT 5', con=engine)
df_check_test.head()

100%|█████████████████████████████████████████████████████████████████████████████| 3263/3263 [00:17<00:00, 188.65it/s]


['tweets_train', 'tweets_test', 'tweets_submission', 'tweets_train_clean', 'tweets_test_clean']


,id,keyword,location,text,text_clean,text_len,word_count,text_lemmatized,keyword_clean,keyword_encoded,location_clean
0,0,no_keyword,unknown,Just happened a terrible car crash,just happened a terrible car crash,34,6,just happen a terrible car crash,no keyword,147,unknown
1,2,no_keyword,unknown,"Heard about #earthquake is different cities, s...",heard about earthquake is different cities sta...,61,9,hear about earthquake be different city stay s...,no keyword,147,unknown
2,3,no_keyword,unknown,"there is a forest fire at spot pond, geese are...",there is a forest fire at spot pond geese are ...,94,19,there be a forest fire at spot pond geese be f...,no keyword,147,unknown
3,9,no_keyword,unknown,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting spokane wildfires,37,4,apocalypse light spokane wildfire,no keyword,147,unknown
4,11,no_keyword,unknown,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kills in china and taiwan,42,7,typhoon soudelor kill in china and taiwan,no keyword,147,unknown
